**MainFlow**

entry

In [ ]:
#set($callId = $params.callId)
#if(!$phone)
 #set($phone = $params.get("session.callerid"))
#end

#set($voiceFileName = "hello_welcome_ai")
`$voiceFileName`

entry-handler

In [ ]:
#set($params.playedMessages = 'Entry-handler')
::::::::::::::::::::::::::::::::
::::: MainFlow params logger::::
phone:::::::::::: $phone
session.callid::: $params.get("session.callerid")
::::::::::::::::::::::::::::::::
params::::::::::: $params
messages::::::::: $messages
::::::::::::::::::::::::::::::::
::::personId::::::::::::::: $personId
::::actorType:::::::::::::: $actorType
::::actorId:::::::::::::::: $actorId
::::refKey::::::::::::::::: $refKey::::messagesscheduledServicegroupId::::phone::::::::::::::::::$phone
::::internal:::::::::::::::internal:::causes exception that hides other watch logs
::::callId = $internal.callId
::::::::::::::::::::::::::::::::

#set($callId = $internal.callId)
#set($pinAttempts = 0)
#set($maxPinAttempt = 3)
#set($isLocaleSet = "NO")
  
#if($refKey)
	#set($sourceId = $refKey)
  #set($sourceType = "SCHEDULED_SERVICE_GROUP")
  #set($serviceGroup = $messagingGroupService.getById($Integer.parseInt($refKey)))
  
  ::::personId::::::::::::::: $personId
::::actorType:::::::::::::: $actorType
::::actorId:::::::::::::::: $actorId
  

  ##set($patientId = $serviceGroup.getPatient().personId)
patientId:::::::: $patientId
service group:::: $sourceId
  ## testing|main
#else
	#set($sourceId = $callId)
  #set($sourceType = "OTHER")
  #set($patientId = $actorId)
#end

#if(!$actorId && $personId)
	#set ($actorId = $personId)
#elseif($actorId)	
	#set ($actorId = $Integer.parseInt($actorId)) 
#end

Testing actorId:::::::::: $actorId
#if($actorId)
	#set($person = $personDAO.getPerson($actorId))
#else
 |main|
#end 


#if($person)

	  #set($personStatus = "DEACTIVATED")
    #set($maxPinAttempt = 3)
    #set($pinAttempts = 0)
  	#foreach($attribute in $person.getActiveAttributes())

      #if($attribute.attributeType.name == 'Person status' && !$attribute.voided)
      	#set($personStatus = $attribute.value)
      #end
      #if($attribute.attributeType.name == 'personLanguage' && !$attribute.voided)
        #set($locale = $attribute.value)
      	#set($isLocaleSet = "YES")
        #set($isvoided = $attribute.voided)
      #end
      ##if($attribute.attributeType.name == 'personLanguage' && !$attribute.voided)
      	##set($locale = $attribute.value)
      ##end
    #end
:::::::::Locale:::::::::::::: $locale
:::::::::isLocaleSet::::::::: $isLocaleSet
:::::::::PersonStatus:::::::: $personStatus
:::::::::isvoided?::::::::::: $isvoided

phone:::::::::::: $phone
##session.callid::: $params.get("session.callerid")
::::::::::::::::::::::::::::::::
##params::::::::::: $params
##messages::::::::: $messages
::::::::::::::::::::::::::::::::
personId::::::::: $personId
actorType:::::::: $actorType
actorId:::::::::: $actorId
refKey::::::::::: $refKey
internal::::::::: $internal
::::::::::::::::::::::::::::::::
		|MainFlow.main|
#else
		|exit|
#end

main

In [ ]:
#set($voiceFileName = "visit_reminder_intro")
`$voiceFileName`

main-handler

In [ ]:
#set($params.playedMessages = 'MainFlow_main')
#set($noInputCount = 0)
#set($invalidCount = 0)

::::check local here:::: If correct then continue to outgoing flow
#if($isLocaleSet == "YES") 
:::: locale set:::: 


  #if($internal.callDirection == "OUTGOING")
    #if($messages.size() == 0)
      ::::Go to outgoing flow whether message exists/not 
      ::::This allows "initiate call" button to move call on to outgoingflow
      :::: outging message size is zero!:::: 
      |OutgoingFlow.entry|
      :::: may fail because outging message size is zero!:::: 
    #else 
      ::::message size is greater than zero:::::::::::::::::::::::::
      :::: outoging message size is greater than 0!:::: 
      |OutgoingFlow.entry|
    #end
  #else
    ::::TODO:::::incoming call:::
  #end
#else
		|goodbye|
#end 

exit

#set($params.playedMessages = 'MainFlow_exit')

no-input

no-input-handler

In [ ]:
#set($params.playedMessages = 'MainFlow_no-input')
#if ($noInputCount < 2)
	#set($noInputCount = $noInputCount + 1)
	|$returnNode|
#else
	|OutgoingFlow.remove-first-message|
#end

goodbye

In [ ]:
#set($voiceFileName = "goodbye")
`$voiceFileName`

goodbye-handler

In [ ]:
#set($params.playedMessages = 'Mainflow_goodbye')
|exit|